<a href="https://colab.research.google.com/github/krmiddlebrook/intro_to_deep_learning/blob/master/machine_learning/mini_lessons/Cleaning_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Data: preparing data for machine learning models
One of the most important skills in data science and machine learning is being able to clean a dataset. This skill enables you to prepare and use data in machine learning models and statistical analysis. In this lesson, we focus on learning the foundations of this skill by cleaning a dataset with various column types (i.e., numerical, categorical, string, date, unique identifier). Specifically, we clean the [Google Play Store dataset](https://www.kaggle.com/lava18/google-play-store-apps) so it can be used later in a machine learning model. 

Here is the roadmap for this lesson:
1. Loading the [Google Play Store dataset](https://www.kaggle.com/lava18/google-play-store-apps)
2. Exploring the dataset
3. Cleaning the data
  - Applying a function to each row of a specific column
  - Filtering data 
  - Manipulating strings in a specific column with Pandas
  - Converting column types
  - Replacing or removing NaNs and dropping duplicate data
  - Making categorical data useable in models by applying "one-hot encoding"
  - Combining data
  - Dropping columns
4. Saving the cleaned dataframe to a csv file

*Note, please run this notebook on the Google Chrome browser so you can download your cleaned dataset easily.*

In [ ]:
# load the necessary libraries
import pandas as pd
import numpy as np

## 1. Loading the Google Play Store dataset
The [Google Play Store dataset](https://www.kaggle.com/lava18/google-play-store-apps) is a web scraped dataset containing data for 10k Play Store apps for analysing the Android market. The dataset has several different column types, and some preprocessing is necessary to make it useable in a machine learning model. 

In this step we focus on loading the data. Specifically, we 1) download the data from Kaggle, 2) upload it to our notebook, and 3) load it into a Pandas dataframe. 

For step 1, we navigate to the Google Play Store dataset on Kaggle [Google Play Store dataset](https://www.kaggle.com/lava18/google-play-store-apps), and download the "googleplaystore.csv" file locally to our computer. 

For step 2, we upload the "googleplaystore.csv" file to our Google Colab notebook. First, click on the folder icon on the left side of the notebook (under the <> icon). Then, select "Upload", navigate to the "googleplaystore.csv" file on your local machine and select it. This process will make the data file accessible in your notebook. 

Finally, load the data into a Pandas dataframe by run the code in the below cell.

In [ ]:
# get this notebook's file path for the "googleplaystore.csv" file by clicking
# on the three vertical dots that appear when you scroll over the file.    
data_url = '/content/googleplaystore.csv' # this path may be different for you

df = pd.read_csv(data_url) # load the data into a pandas dataframe

# 2. Exploring the dataset
Once you've loaded the dataset into a Pandas dataframe, you want to spend some time exploring the data to get a sense of what it looks like. 

In [ ]:
# look at the first 5 rows of data
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [ ]:
# check the column types and get basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


By viewing the first 5 rows of our data it looks like there are several categorical columns, various numerical columns, and one date column. But, the `info()` function indicates that our dataframe thinks most of the columns contain an object type (i.e., strings or complex objects), and only one numeric column. In the next step we will address this difference.   

## 3. Cleaning the data
To make our dataset useful for most machine learning models, we need to convert the columns to the correct data types, remove errors, and one-hot encode the categorical data.

#### Applying a function to each row of a specific column
We will apply a function to scale and clean the "Size" column. There are several ways to do this, here we use the pandas `map` method, which applies our function row-wise to the data in a specific column.

In [ ]:
#scaling and cleaning "Size" (of app) column
def change_size(size):
    if 'M' in size:
        x = size[:-1]
        x = float(x)*1000000
        return(x)
    elif 'k' == size[-1:]:
        x = size[:-1]
        x = float(x)*1000
        return(x)
    else:
        return None

df["Size"] = df["Size"].map(change_size)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            9145 non-null   float64
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(2), object(11)
memory usage: 1.1+ MB


Notice that the Dtype of the Size column as changed from an object to a float64 (i.e., numeric type).  

#### Filtering data
After looking at the dataset and trying to convert some columns to numeric types, I realized that there was at least one row with erroneous data. The below code is how I found it and removed it from the dataset. In particular, I filtered the data by looking for rows that did not contain the letter 'M' in the Reviews column. This only removed 1 row (i.e., the erroneous one). The fewer rows we remove from the original dataset the better!   

In [ ]:
# identify the erroneous data and remove it from the dataset

# here I use the the "Reviews" column to find rows that *do* contain the string 'M'
print(df.loc[df.Reviews.str.contains('M'), :])

# here I remove those rows from the dataset by *not* including the rows in the "Reviews" column that have the string 'M'
df = df.loc[~df.Reviews.str.contains('M'), :]

                                           App Category  ...  Current Ver Android Ver
10472  Life Made WI-Fi Touchscreen Photo Frame      1.9  ...   4.0 and up         NaN

[1 rows x 13 columns]


#### Manipulating strings in a specific column with Pandas
In this step we use the Pandas column string methods to clean a column with strings. Specifically, we prepare the Installs column to be converted into a numeric column by replacing the ',' and '+' symbols.  

In [ ]:
# clean and convert the "Installs" column to int
df['Installs'] = df['Installs'].str.replace(',', '')
df['Installs'] = df['Installs'].str.replace('+', '')

# remove mislabeled row by filtering for rows that contain more than just numbers
df = df.loc[~df['Installs'].str.contains('\D+'), :].reset_index(drop=True)


#### Converting column types
There are several wells to convert the data type of a pandas dataframe column. In this example, we will use the `astype` method to convert the 'Installs' column we just cleaned into an integer type.

In [ ]:
# convert the Installs column to a numeric data type column
df['Installs'] = df['Installs'].astype(int)

# lets also convert other columns we cleaned to numeric data type columns
df['Reviews'] = df['Reviews'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10840 entries, 0 to 10839
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10840 non-null  object 
 1   Category        10840 non-null  object 
 2   Rating          9366 non-null   float64
 3   Reviews         10840 non-null  int64  
 4   Size            9145 non-null   float64
 5   Installs        10840 non-null  int64  
 6   Type            10839 non-null  object 
 7   Price           10840 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10840 non-null  object 
 11  Current Ver     10832 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 1.1+ MB


#### Replacing or removing NaNs and dropping duplicate data
In this step we will replace the NaN entries in the numeric columns with the average colum values. In addition, we will remove any rows that contain the same data. 

In [ ]:
# Note, there are several ways to replace values, here I use the fillna method
# create dictionary where the keys are the numeric column names and the values are their average values
fillers = {'Rating': df.Rating.mean(), 'Reviews': df.Reviews.mean(), 'Size': df.Size.mean(), 'Installs': df.Installs.mean()}
df = df.fillna(value=fillers)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10840 entries, 0 to 10839
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10840 non-null  object 
 1   Category        10840 non-null  object 
 2   Rating          10840 non-null  float64
 3   Reviews         10840 non-null  int64  
 4   Size            10840 non-null  float64
 5   Installs        10840 non-null  int64  
 6   Type            10839 non-null  object 
 7   Price           10840 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10840 non-null  object 
 11  Current Ver     10832 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 1.1+ MB


Notice that the Non-Null Count in the numeric column has changes. 

#### Making categorical data useable in models by applying "one-hot encoding"
In this step we learn how to make categorical variables useable in machine learning models by "one-hot encoding" their values. Like previous steps, there are several ways this can be accomplished, here I use the pandas `get_dummies` method. 

In [ ]:
# converting non-numeric columns to numeric columns using "one-hot encoding"
catgry = pd.get_dummies(df['Category'],prefix='catg')
cr = pd.get_dummies(df['Content Rating'],prefix='cr')

catgry.head()

,catg_ART_AND_DESIGN,catg_AUTO_AND_VEHICLES,catg_BEAUTY,catg_BOOKS_AND_REFERENCE,catg_BUSINESS,catg_COMICS,catg_COMMUNICATION,catg_DATING,catg_EDUCATION,catg_ENTERTAINMENT,catg_EVENTS,catg_FAMILY,catg_FINANCE,catg_FOOD_AND_DRINK,catg_GAME,catg_HEALTH_AND_FITNESS,catg_HOUSE_AND_HOME,catg_LIBRARIES_AND_DEMO,catg_LIFESTYLE,catg_MAPS_AND_NAVIGATION,catg_MEDICAL,catg_NEWS_AND_MAGAZINES,catg_PARENTING,catg_PERSONALIZATION,catg_PHOTOGRAPHY,catg_PRODUCTIVITY,catg_SHOPPING,catg_SOCIAL,catg_SPORTS,catg_TOOLS,catg_TRAVEL_AND_LOCAL,catg_VIDEO_PLAYERS,catg_WEATHER
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Notice that the columns now represent the category type and the rows represent whether the data belongs to a particular category type. Another way to think of this one-hot encoding process is to consider that each row is converted into a vector that contains mostly 0's with a 1 at the index that corresponds to the category type that the data belongs to. 

#### Combining data
Now that we have converted the categorical data into one-hot encoded dataframes, we need to combine them. In particular, we want to combine them with the original dataset. Here I use the Pandas `concat` method.  

In [ ]:
print('original dataframe shape', df.shape)

# create an array containing the original dataframe and the two one-hot encoded dataframes
frames=[df,catgry,cr]
# combine them to create a new dataframe
df=pd.concat(frames,axis=1)
print('combined dataframe shape', df.shape)

original dataframe shape (10840, 13)
combined dataframe shape (10840, 52)


#### Dropping columns
Now that we've combined the data, we need to drop some now unnecessary columns. Here we use the Pandas `drop` method.

In [ ]:
df = df.drop(['Category', 'Content Rating'], axis=1)


## 4. Saving the cleaned dataframe to a csv file
Now that our data is clean, we should save it so that we can quickly load the cleaned data into a dataframe that can be easily analyzed or passed to a machine learning model. Here we use the Pandas `to_csv` method to write (i.e., save) the dataframe to a csv file.

In [ ]:
# save the file on the colab hosting maching (i.e., the remote computer)
save_path = './cleaned_googleplaystore.csv'
df.to_csv(save_path, index=False)

# to download hosted files to your local machine when using colab run this
from google.colab import files
files.download(save_path) 